# Video Game Review Analysis: Critics vs. Users
### A Statistical Study of Metacritic, IGN, and OpenCritic Trends

This notebook explores the relationship between professional critics and general players. We investigate "review inflation," scoring biases, and identify the most controversial titles in gaming history.

## 1. Data Loading
The loading of the dataset and initial environment setup.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel, levene, pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

try:
    ign = pd.read_csv('Databases/IGN_data.csv')
    meta = pd.read_csv('metacritic_pc_games.csv')
    oc = pd.read_csv('Opencritic_dataset.csv')
except FileNotFoundError:
    ign = pd.read_csv('IGN_data.csv')
    meta = pd.read_csv('metacritic_pc_games.csv')
    oc = pd.read_csv('Opencritic_dataset.csv')

## 2. Preprocessing & Normalization
Standardizing column names and scaling scores to a 0-100 range.

In [ ]:
for df in [ign, meta, oc]:
    df.columns = df.columns.str.strip()

meta.rename(columns={'Game Title':'game', 'Overall Metascore':'critic_score', 
                     'Overall User Rating':'user_score', 'Game Release Date':'date'}, inplace=True)
oc.rename(columns={'Title':'game', 'Score':'critic_score_oc', 'Release Date':'date_oc'}, inplace=True)
ign.rename(columns={'game':'game', 'score':'critic_score_ign', 'released_date':'date_ign'}, inplace=True)

meta['date'] = pd.to_datetime(meta['date'], errors='coerce')
oc['date_oc'] = pd.to_datetime(oc['date_oc'], errors='coerce')
ign['date_ign'] = pd.to_datetime(ign['date_ign'], errors='coerce')

merged = meta.merge(oc, on='game', how='outer').merge(ign, on='game', how='outer')

for col in ['critic_score', 'user_score', 'critic_score_oc', 'critic_score_ign']:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

merged['user_score'] = merged['user_score'] * 10
merged['critic_score_ign'] = merged['critic_score_ign'] * 10

merged['year'] = merged['date'].dt.year.fillna(merged['date_oc'].dt.year).fillna(merged['date_ign'].dt.year)
merged_clean = merged.dropna(subset=['year']).copy()

## 3. Hypothesis Testing
Validating trends using Pearson's Correlation and checking for bias via Paired T-Tests.

In [ ]:
test_results = []

df_h1 = merged_clean.dropna(subset=['critic_score', 'year'])
corr_meta, p_meta = pearsonr(df_h1['year'], df_h1['critic_score'])
test_results.append({'Hypothesis': 'H1: Critics vs Year', 'Stat': corr_meta, 'P-Value': p_meta})

df_h2 = merged_clean.dropna(subset=['critic_score', 'user_score'])
t_stat, p_val_h2 = ttest_rel(df_h2['critic_score'], df_h2['user_score'])
test_results.append({'Hypothesis': 'H2: Critic vs User Mean', 'Stat': t_stat, 'P-Value': p_val_h2})

stat_var, p_val_var = levene(df_h2['user_score'], df_h2['critic_score'])
test_results.append({'Hypothesis': 'H4: Variance Comparison', 'Stat': stat_var, 'P-Value': p_val_var})

summary_df = pd.DataFrame(test_results)
summary_df['Significant?'] = summary_df['P-Value'] < 0.05
display(summary_df)

## 4. Visualizing the Data
Generating density plots and trend lines to visualize the Critic-User gap.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.kdeplot(merged_clean['critic_score'], label='Critics (Meta)', fill=True, ax=axes[0])
sns.kdeplot(merged_clean['user_score'], label='Users (Meta)', fill=True, ax=axes[0])
axes[0].set_title('Density of Review Scores')
axes[0].legend()

yearly_trends = merged_clean.groupby('year')[['critic_score', 'user_score']].mean()
sns.lineplot(data=yearly_trends, ax=axes[1])
axes[1].set_title('Average Review Scores Over Time')

plt.show()

## 5. Controversy Analysis
Using a custom function to identify the top games with the highest discrepancy between groups.

In [ ]:
def top_discrepancies_per_year(df, n=5):
    paired = (
        df.dropna(subset=['critic_score', 'user_score', 'year', 'game'])
        .groupby(['year', 'game'], as_index=False)
        .agg({'critic_score': 'mean', 'user_score': 'mean'})
    )
    paired['score_diff'] = paired['critic_score'] - paired['user_score']
    paired['abs_diff'] = paired['score_diff'].abs()
    
    return (
        paired.sort_values(['year', 'abs_diff'], ascending=[True, False])
        .groupby('year').head(n)
        .reset_index(drop=True)
    )

controversy_df = top_discrepancies_per_year(merged_clean)
display(controversy_df.head(10))

### Final Summary
**Key Findings:**
1. **Bias Detection:** Professional critics consistently score games higher than users (statistically significant via T-Test).
2. **Polarization:** Users exhibit significantly higher variance, indicating more frequent use of extreme scores (0 or 100) compared to critics.
3. **Trends:** Review scores have remained relatively stable over the last decade, with minor fluctuations during major console releases.